In [6]:
# This is the program for making a value for authorship

import nltk

import os
import re
import pandas as pd

# Because I use Jupyter notebook, I need to escape the case that there is .ipynb_checkpoint in a Textdata directory.
# by using re, files is the name of file that is in Textdata directory, also that end wit .(txt).
files = [ i for i in os.listdir("./Data/Textdata/") if re.match('.*(.txt)$', i)]
files.sort()
document = []

for f in files:
    docs = open("./Data/Textdata/"+f, "r").read()
    document.append(docs)

words = []
Pos = []

for i in range(len(document)):
    words.append(nltk.word_tokenize(document[i]))
    Pos.append(nltk.pos_tag(words[i]))

PerfectV = []
NonPerfectV = []
Num_Verbs = []

for i in range(len(document)):    
    
    v_p = []
    v_np = []
    k = 0 
    
    for j in range (len(Pos[i])):
        if k != 0:
            k -= 1
            continue
            
        if re.match("have|has|had", Pos[i][j][0].lower()):
            if Pos[i][j+1][1] == 'VBN' or Pos[i][j+1][0] == 'had':
                v_p.append(Pos[i][j+1][0])
                if Pos[i][j+1][0] == 'had':
                    k = 1
            elif Pos[i][j+2][1] == 'VBN' or Pos[i][j+2][0] == 'had':
                v_p.append(Pos[i][j+2][0])
                if Pos[i][j+2][0] == 'had':
                    k = 2
            else:
                v_np.append(Pos[i][j][0])
        elif Pos[i][j][1].startswith('VB') and Pos[i][j][1] != 'VBN' and Pos[i][j][1] != 'VBG':
            v_np.append(Pos[i][j][0])
    
    PerfectV.append(len(v_p))
    NonPerfectV.append(len(v_np))
    Num_Verbs.append(len(v_p)+len(v_np))

author = pd.read_csv("./Data/Authordata/author.csv", index_col=0)
    
table = pd.DataFrame({"PerfectV": PerfectV,
                     "NonPerfectV":NonPerfectV,
                     "Num_Verbs":Num_Verbs}, index=files)

table['featureValue'] = (table['PerfectV']-table['NonPerfectV'])/(table['Num_Verbs']+1) # for escaping divide by 0.

V = []
for i in range(len(author)):
    V.append(author['Author'][i])

table['Author'] = V

print(table)

table.to_csv("texts.csv")

            PerfectV  NonPerfectV  Num_Verbs  featureValue Author
text01.txt         0            1          1     -0.500000      A
text02.txt         0            1          1     -0.500000      A
text03.txt         0            1          1     -0.500000      A
text04.txt         0            1          1     -0.500000      A
text05.txt         0            1          1     -0.500000      A
text06.txt         0            1          1     -0.500000      A
text07.txt         0            1          1     -0.500000      A
text08.txt         0            0          0      0.000000      A
text09.txt         0            1          1     -0.500000      A
text10.txt         0            1          1     -0.500000      A
text11.txt         1            0          1      0.500000      B
text12.txt         1            0          1      0.500000      B
text13.txt         0            1          1     -0.500000      B
text14.txt         0            1          1     -0.500000      B
text15.txt

In [14]:
# This is a program for Knn

import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

texts = pd.read_csv("texts.csv", index_col = 0)
texts = texts.sample(frac=1, random_state=0)

X = texts.loc[:,['featureValue']]
Y = []

for i in range(len(texts)):
    Y.append(texts['Author'][i])
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.1,random_state=3)

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

/Users/gotoyagi/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [18]:
sentence = "I have had a dog for 6 years."

words2 = nltk.word_tokenize(sentence)
pos2 = nltk.pos_tag(words2)
v_p2 = []
v_np2 = []
num = []
flag = 0

for j in range(len(pos2)):
    if flag != 0:
        flag -= 1
        continue
    if re.match("have|has|had", pos2[j][0].lower()):
        if pos2[j+1][1] == 'VBN' or pos2[j+1][0] == 'had':
            v_p2.append(pos2[j+1][0])
            if pos2[j+1][0] == 'had':
                flag = 1
        elif pos2[j+2][1] == 'VBN' or pos[j+2][0] == 'had':
            v_p2.append(pos2[j+2][0])
            if pos2[j+2][0] == 'had':
                flag = 2
        else:
            v_np2.append(pos2[j][0])
    elif pos2[j][1].startswith('VB') and pos2[j][1] != 'VBN' and pos2[j][1] != 'VBG':
        v_np2.append(pos2[j][0])
        
num.append(len(words2))
print(v_p2, v_np2)
P2 = []
P2.append(len(v_p2))

NP2 = []
NP2.append(len(v_np2))

NV2 = []
NV2.append(len(v_np2) + len(v_p2))

table2 = pd.DataFrame({"PerfectV":P2,
                      "NonPerfectV":NP2,
                      "Num_Verbs":NV2})

table2['featureValue'] = (table2["PerfectV"] + table2["NonPerfectV"]) / (table2["Num_Verbs"]+1)
print(table2.loc[:,['featureValue']])

['had'] []
   featureValue
0           0.5


In [1]:
# Train by RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

pred2_Y = rf.predict(table2.loc[:,['featureValue']])

print(pred2_Y,Y_test)

print(rf.score(X_test, Y_test))



NameError: name 'X_train' is not defined

In [1]:
%run main_RFver.py

            Num_Place  Num_Time  Num_ad  PerfectV  NonPerfectV  Num_Verbs  \
text01.txt          0         0       0         0            1          1   
text02.txt          0         1       1         0            1          1   
text03.txt          0         0       0         0            1          1   
text04.txt          0         0       0         0            1          1   
text05.txt          0         0       0         0            1          1   
text06.txt          0         1       1         0            1          1   
text07.txt          0         0       0         0            1          1   
text08.txt          0         0       0         0            0          0   
text09.txt          1         1       2         0            1          1   
text10.txt          0         1       1         0            1          1   
text11.txt          0         1       1         1            0          1   
text12.txt          0         0       0         1            0          1   